# Primera entrega
Luis Montenegro - 21699<br>
Javier Prado - 21486<br>
Bryan España<br>
Ángel Herrarte<br>


## Limpieza de datos
En esta entrega se hará una recolección y limpieza de datos solamente. No habrá modelado ni análisis. 
<br>Solamente velar por la integridad, coherencia y cohesión del conjunto de datos.facilidad de manejo.

In [ ]:
# !pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [15]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from collections import defaultdict

### Conversión de .xls a .csv

Debido a que los archivos crudos descargados desde la página del Mineduc vienen en formato .xls formateado como .html tenemos que hacer cambio de eso.<br>
Esto por múltiples razones:
- Mejorar la estructura de los datos.
- Preservar solamente la información requerida (los .xls almaceban más páginas e información irrelevante)
- Mayor falicidad de usar y obtener los datos si están en formato delimitado por comas.
- Tener datos más ligeros ya que no acarreamos con mucha información innecesaria. 

In [16]:
def transform_html_to_csv(input_path: str, output_path: str=None, tables_to_convert: list[int] = [9]) -> None:
    '''
    Turns .html files into .csv files

    Params:
        input_path: where the .html files are stored
        output_path: where the .csv will be stored
    Returns:
        out: None
    '''
    if output_path is None:
        output_path = input_path 
    else:
        os.makedirs(output_path, exist_ok=True)

    for filename in tqdm(os.listdir(input_path), desc='Converting Files'):
        if filename.endswith('.xls'):
            html_path:str = os.path.join(input_path, filename)
            base_name:str = os.path.splitext(filename)[0]
            # try to read the excel and copy it into a .csv file
            try: 
                tables:list[pd.DataFrame] = pd.read_html(html_path)
                # if no tables, skip
                if not tables:
                    print(f"No tables found in '{filename}'")
                    continue

                # convert each of the tables selected to .csv
                for i, df in enumerate(tables):
                    if i in tables_to_convert:
                        csv_filename = f"{base_name}_table{i}.csv"
                        csv_path = os.path.join(output_path, csv_filename)
                        df.to_csv(csv_path, index=False)
                        print(f"Converted '{filename}' to '{csv_filename}'") # print all of the transformations of the .xls to its respective .csv files
                    
            except Exception as e:
                print(f"Error processing file '{filename}' : {e}")

def remove_files(path: str, extension: str) -> None:
    '''
    Removes all files that have a certain extension
    Input:
        path: folder path where .xls to be removed are stored
    '''
    file_list: list[str] = os.listdir(path)
    if not file_list:
        print(f"No files removed since there was none found: '{path}'")
        return

    for filename in file_list:
    
        if filename.endswith(extension):
            try:
                xls_path = os.path.join(path, filename)
                os.remove(xls_path)
            except Exception as e:
                print(f"Error removing file '{filename}' : {e}")


In [17]:
# start by turning .xls files to .csv files
transform_html_to_csv(input_path="../Dataset_raw", output_path="../Dataset_cleaned", tables_to_convert=[9])

Converting Files: 0it [00:00, ?it/s]


In [18]:
# remove .xls files if necessary
remove_files(path="../Dataset_raw", extension='.xls')

No files removed since there was none found: '../Dataset_raw'


### Estandarizar la estructura de los .csv
Es importante que los .csv con los que estamos trabajando tengan una misma estructura. Por lo tanto, debemos asegurarnos que todos sean iguales. <br>
Esto lo hacemos con la finalidad de evitar errores al limpiar los datos más adelante o al concatenarlos al final. 

Queremos que todos los .csv tengan 17 títulos de columnas en orden tipo:<br>
CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL<br>

Otra cosa a considerar es eliminar la primera fila y reajustar índices. Debido a que la primera fila son puros números en vez de los nombres de las columnas. Al igual que debemos
eliminar la última fila que es todos nulos. 

Para los que tenían error de formato o formato diferente solamente copiamos los datos en un .csv ya que usualmente eso se debe a que solamente había 1 establecimiento encontrado.


In [19]:
COLUMNS = ["CODIGO","DISTRITO","DEPARTAMENTO","MUNICIPIO","ESTABLECIMIENTO","DIRECCION","TELEFONO","SUPERVISOR","DIRECTOR","NIVEL","SECTOR","AREA","STATUS","MODALIDAD","JORNADA","PLAN","DEPARTAMENTAL"]

def clean_headers_and_trailers(path: str) -> None:
    '''
    Removes first row and reindexes the data within.
    Also removes the last row that contains only nulls
    '''
    for file in tqdm(os.listdir(path), desc="Converting files"):
        if file.endswith('.csv'):
            try:
                f_path = os.path.join(path, file) # path to the file
                df: pd.DataFrame = pd.read_csv(f_path)
                
                # drop first row and reindex if the first row are digits and not columns
                if all(col.strip().isdigit() for col in df.columns):
                    df.columns = df.iloc[0]
                    df = df[1:].reset_index(drop=True)

                # remove last row if only nulls are found
                if df.tail(1).isnull().all(axis=1).iloc[0]:
                    df = df.iloc[:-1]
                    print(f"Removed last row: {file}")
                
                df.to_csv(f_path, index=False)  
                
                print(f"Finished: {file}")
            
            except Exception as e:
                print(f"Error processing '{file}': {e}")


In [ ]:
# clean headers and trailers
# clean_headers_and_trailers(path = "../Dataset_cleaned")

### Plan para limpieza de datos
Posteriormente a la transformación de formato y filtración a solo datos crudos de nuestro interés, procederemos a limpiar los datos en sí. <br>
1. Verificaremos si existe datos NA o Null en los archivos. Llenaremos los datos faltantes con datos artificiales ya que no podemos eliminar ningún registro <br>
debido a que si eliminamos alguno, sería una sede faltante. Cosa que afectaría de gran manera el conjunto de datos.
2. Homogenizaremos la información. Es decir, que todos los archivos tengan el mismo formato i.e. todos los nombres estén escritos igual, los apellidos que tienen tildes se escriban igual en cada archivo,
que las palabras vengan o solo en mayúsculas, solo en minúsculas, etc.
3. Identificaremos las columnas que más trabajo de reajuste necesiten.
- Nombres (Supervisor, Director, Establecimiento, Sector): debido a que puede que exista apellidos iguales pero escritos distinto y eso genere problemas a la hora de verlos. Digamos, puede que Hernández aparezca con tilde en unos registros y en otros no. También es bueno verificar que estén escritos todos o en mayúsculas o en minúsculas (o todos iguales), ya que digamos, si tenemos Santa rosa en un registro y Santa Rosa en otro, a la hora de hacer un encoding estos resultarán con dos valores distintos. 
- Dirección: Asegurarnos que las direcciones lleven una estructura similar.
- Códigos: Verificar que los códigos de los registros sean únicos y evitar tener repetidos.



El conjunto de datos corresponde a los establecimientos educativos de Guatemala que llegan hasta el nivel diversificado. Está organizado en 85 archivos CSV, uno por cada departamento y municipio del país.

Cada archivo contiene información detallada de cada establecimiento, como el nombre del centro, su ubicación, datos de contacto, modalidad de enseñanza y otros datos administrativos.


Los archivos csv estan clasificados por departamentos de Guatemala. Las variables que contiene esta dataset son las siguientes sumando un total de 17 variables a analizar.

- CODIGO: código único del establecimiento

- DISTRITO: distrito educativo

- DEPARTAMENTO: nombre del departamento

- MUNICIPIO: municipio donde se ubica

- ESTABLECIMIENTO: nombre del centro educativo

- DIRECCION: dirección física

- TELEFONO: número de contacto

- SUPERVISOR: nombre del supervisor

- DIRECTOR: nombre del director

- NIVEL: nivel educativo (ej. Básico, Diversificado)

- SECTOR: sector oficial o privado

- AREA: área urbana o rural

- STATUS: estado del centro (ej. Abierta)

- MODALIDAD: modalidad lingüística (ej. Monolingüe, Bilingüe)

- JORNADA: jornada de estudio (ej. Matutina, Vespertina)

- PLAN: plan educativo (ej. Diario, Fin de semana)

- DEPARTAMENTAL: nombre del departamento de adscripción

### Encontrar Null o NA en cualesquiera archivos
Comenzaremos viendo si existe cualesquiera archivos con datos faltantes.

In [20]:
dataset_path: str = "../Dataset_cleaned" # define folder path as a variable for easier handling
report_path: str = "../Data_null_report"

def get_null_stats(path:str) -> None:
        '''
        Prints the columns with most null counts and the files with most nulls
        '''
        df: pd.DataFrame = pd.read_csv(path)
        print("\n Top columns with the most nulls:")
        top_columns = df.groupby('column')['nulls'].sum().sort_values(ascending=False).head(10)
        print(top_columns)

        print("\n Files with the most nulls:")
        top_files = df.groupby('file_name')['nulls'].sum().sort_values(ascending=False).head(10)
        print(top_files)

def count_null_instances(path:str, report_name:str = "null_report.csv") -> None:
    '''
    Counts how many missing instances are found per column per .csv and saves it into a .csv
    '''
    files: list[str] = os.listdir(path=path)
    report_file: str = os.path.join(report_path, report_name)
    os.makedirs(name=report_path, exist_ok=True) # make a new directory to save this information
    data:list[dict] = []
    for file in tqdm(files, desc="Counting nulls.."):
        file_path:str = os.path.join(path, file)
        try:
            df: pd.DataFrame = pd.read_csv(file_path) 
            null_counts = df.isnull().sum() # register nulls found in the df per column
            for col, count in null_counts.items():
                data.append({
                    "file_name": file,
                    "column": col,
                    "nulls": count
                })
        
        except Exception as e:
            print(f"Error processing {file} : {e}")
    
    report_df: pd.DataFrame = pd.DataFrame(data=data)
    report_df.to_csv(report_file, index=False) # not necessary to save it as a .csv



In [20]:
count_null_instances(path=dataset_path) # count nulls found per column in each .csv

Counting nulls..: 100%|██████████| 85/85 [00:00<00:00, 429.89it/s]


In [21]:
get_null_stats(path="../Data_null_report/null_report.csv")


 Top columns with the most nulls:
column
TELEFONO           4320
DIRECTOR            116
DIRECCION            23
AREA                  0
DEPARTAMENTAL         0
DEPARTAMENTO          0
DISTRITO              0
ESTABLECIMIENTO       0
CODIGO                0
JORNADA               0
Name: nulls, dtype: int64

 Files with the most nulls:
file_name
alta_verapaz_primaria_table9.csv        633
alta_verapaz_preprimaria_table9.csv     376
quiche_primaria_table9.csv              334
peten_primaria_table9.csv               311
jutiapa_primaria_table9.csv             277
huehuetenango_primaria_table9.csv       258
san_marcos_primaria_table9.csv          225
santa_rosa_primaria_table9.csv          225
baja_verapaz_primaria_table9.csv        166
huehuetenango_preprimaria_table9.csv    138
Name: nulls, dtype: int64


Como podemos observar, las columnas con mayor recuento de nulos son de TELEFONO, DIRECTOR y DIRECCION. Mientras que los archivos con más nulos son altaverapaz con primaria y preprimaria. 
Por lo tanto, lo que debemos hacer con esto es darles un valor a los nulos. Por ejemplo, un número predeterminado cuando no hay, o un texto único para cuando no haya nombres o direciones, etc. <br>
De esta manera no eliminamos ni afectamos el conjunto de datos y aún logramos clasificarlos dentro de su propia categoría.

### Verificación de tipos de datos y consistencia entre archivos
Antes de comenzar a llenar los campos vacíos, debemos asegurarnos que los campos que estamos llenando contengan los mismos tipos. <br>
Es decir, que no tengamos archivos .csv que tengan tipos int64 y otros object bajo la misma columna. Ya que esto generará problemas a la hora<br>
de trabajar los datos y puede generar dificultades de manejo.


In [33]:
# check datatypes for each column within a file
def check_datatypes(file: str, examples: bool = False) -> tuple[defaultdict[set], defaultdict[dict]]:
    df: pd.DataFrame = pd.read_csv(file)
    type_map = defaultdict(set)
    examples_map = defaultdict(dict)
    for col in df.columns:
        dtype = str(df[col].dtype)
        type_map[col].add(dtype)
        
        if examples:
            for val in df[col]:
                if pd.notnull(val):
                    val_type = str(pd.Series([val]).dtype)
                    if val not in examples_map[col]:
                        examples_map[col][val_type] = val
                    break
    return type_map, examples_map

def check_datatype_differences(path: str, examples: bool = False) -> None:
    '''
    Checks if there is any difference between columns' datatypes across all files
    '''
    
    global_type_map = defaultdict(set)
    global_examples_map = defaultdict(dict)
    for file in os.listdir(path):
        if file.endswith('.csv'):
            try: 
                f_path: str = os.path.join(path, file)
                type_map, examples_map = check_datatypes(f_path, examples=examples)
                for col, types in type_map.items():
                    global_type_map[col].update(types)
                
                if examples:
                    for col, example_dict in examples_map.items():
                        for dtype, val in example_dict.items():
                            if dtype not in global_examples_map[col]:
                                global_examples_map[col][dtype] = val

            except Exception as e:
                print(f"Unexpected error on file '{file}': {e}")

    print(f"Columns and their types found")
    for col, types in global_type_map.items():
        print(f"- {col} : {types}")
        if examples and len(types) > 1:
            for dtype, value in global_examples_map[col].items():
                print(f"{dtype} : {repr(value)}")

In [34]:
check_datatype_differences(path=dataset_path, examples=True)

Columns and their types found
- CODIGO : {'object'}
- DISTRITO : {'object'}
- DEPARTAMENTO : {'object'}
- MUNICIPIO : {'object'}
- ESTABLECIMIENTO : {'object'}
- DIRECCION : {'object'}
- TELEFONO : {'float64', 'object', 'int64'}
object : '57101061'
float64 : 51904421.0
int64 : 77641157
- SUPERVISOR : {'object'}
- DIRECTOR : {'object'}
- NIVEL : {'object'}
- SECTOR : {'object'}
- AREA : {'object'}
- STATUS : {'object'}
- MODALIDAD : {'object'}
- JORNADA : {'object'}
- PLAN : {'object'}
- DEPARTAMENTAL : {'object'}


Como podemos observar arriba, tenemos que los tipos encontrados son en su mayoría object, pero la columna TELEFONO posee tanto object como int64 y float 64. <br>
Eso es un problema debido a que no podemos manejar esos 3 tipos a lo largo y ancho del conjunto de datos. Por lo que será necesario indagar a fondo de por qué aparece
esos tipos en el área de teléfono. 

In [ ]:
# TODO change datatypes to appropiate types depending on the columns

### Asignar valores a los datos inexistentes o nulos
Ahora lo que haremos, como se dijo previamente, es asignarle valores a las celdas nulas. <br>
Los siguientes valores predeterminados para las celdas nulas serán los siguientes:<br>

DIRECCION : DESCONOCIDO<br>
TELEFONO : 00000000<br>
DIRECTOR : DESCONOCIDO<br>

Pero primero observaremos los tipos de cada columna para tener mejor definición de qué hay en cada una y que no exista<br>
discrepancia entre los tipos a lo largo de las columnas y archivos.

In [ ]:
# fill missing data with predetermined values
val_dict: dict = {"DIRECCION":"DESCONOCIDO", "TELEFONO": 00000000, "DIRECTOR": "DESCONOCIDO"} 

def fill_nulls_with_values(path:str, value_dict: dict) -> None:
    '''
    Changes null instances within columns and fills them with predetermined values
    '''